<a href="https://colab.research.google.com/github/JuanOcampo29/Finanzas/blob/main/Trabajo_Final_Maria_Lizarazo%2C_Juan_Ocampo_y_Juan_Moncada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Realizado por: María Paula Lizarazo, Juan Pablo Moncada y Juan Alejandro Ocampo


#**Descripción del Problema**
A partir de los datos obtenidos se establece que la empresa XYZ necesita 
optimizar su cadena de distribución en 5 puntos de venta (municipios) diferentes para satisfacer la demanda.

En primera medida, las empresas buscan optimizar los costos para cubrir la demanda y con ello mejorar la rentabilidad por medio del ajuste de los gastos, de esta manera se evitan los sobrecostos lo cual permite mantener una producción optima y una correcta gestión de los recursos. Por otro lado, el aumento de la eficiencia operativa de la empresa logra reducir costos y aumentar la rentabilidad, siendo este el objetivo principal de la empresa, hacer de sus procesos más eficientes. Esto último estructura el problema a solucionar debido a que se busca optimizar el proceso, para reducir los costos de producción mediante las economías de escala, al igual que, buscar lugares centralizados para la producción con el fin de reducir al máximo los costos de transporte.

Por otro lado, tener una producción lo más optima posible permite mejorar la competitividad de la empresa al ajustar los precios para la demanda existente, y generar liquidez para desarrollar proyectos de innovación de maquinaria y capacidad del personal para hacer más eficiente la empresa, aumentando sus posibilidades de perdurar en el tiempo. Esto último implica que la empresa podría soportar una posible crisis económica debido a que sus costos y estructura empresarial son capaces de sobrellevar una baja temporal en los ingresos.

La solución del problema planteado se desarrollará desde dos puntos principales. En el primero se establecerá la operación de la empresa, es decir, cuantas plantas de producción de deben abrir y que tamaño deben tener estas, con el fin de minimizar los costos. De acuerdo con la demanda de la empresa y de cada planta, eso se hará mediante un proceso de optimización CVX, donde también se tendrán en cuenta los costos que se asumirían por tener una planta de más de alta capacidad, recompensado en el nivel de eficiencia (en la producción) de aquello que se logra aportar como valor agregado en comparación de una planta de baja capacidad. Esos costos extra se refieren a mayores costos fijos por la infraestructura necesaria para abrir una de estas plantas y los costos de transporte más altos debido a las distancias que deberían recorrer para llegar a los puntos de distribución de los productos.

Como segundo punto se buscará estimar la demanda en los próximos meses mediante un proceso de estimación de Monte Carlo, con el fin de establecer que la inversión a realizar sea sostenible en el tiempo de acuerdo con la demanda de cada uno de los municipios de operación de la compañía.


#**Objetivos**

**General:**
* Optimizar la cadena de producción y distribución de la empresa XYZ mediante la herramienta LinProg de Python para reducir costos y mejorar rentabilidades.

**Específicos:**

* Determinar el número de óptimo de plantas que se deben abrir y el tamaño de estas para satisfacer la demanda de la empresa XYZ.

* Estimar la demanda de la empresa XYZ para la toma de decisiones con el fin de conseguir la sostenibilidad de esta a largo plazo.




In [1]:
# Importamos las librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import linprog
import cvxpy as cp
import warnings
import cvxpy as cp
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm
import warnings

Demanda y costos de las plantas:

|   | Demanda | A | B | C | D | E | AltoC | BajoC |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| A | 145.4  | \$ 8 | \$ 14 | \$ 21 | \$ 21 | \$ 12 | \$ 4730 | \$ 3230 |
| B | 84.1   | \$14 | \$ 6  | \$ 13 | \$ 14 | \$ 13 | \$ 7270 | \$ 4980 |
| C | 156.4  | \$23 | \$ 13 | \$ 8  | \$ 10 | \$ 22 | \$ 3080 | \$ 2110 |
| D | 1676.8 | \$21 | \$ 14 | \$ 9  | \$ 3  | \$ 20 | \$ 9100 | \$ 6230 |
| E | 2719.6 | \$12 | \$ 13 | \$ 17 | \$ 20 | \$ 6  | \$ 9500 | \$ 6500 |

In [2]:
# Inputs del problema de optimización
I = np.array([1, 2, 3, 4, 5])                       # Puntos de demanda
J = np.array([1, 2, 3, 4, 5])                       # Plantas
d = np.array([145.4, 84.1, 156.4, 1676.8, 2719.6])  # Demandas a satisfacer
AC = np.array([4730, 7270, 3080, 9100, 9500])       # Costo fijo de cada planta con Alta capacidad
BC = np.array([3230, 4980, 2110, 6230, 6500])       # Costo fijo de cada planta con Baja capacidad
MAC = np.array([1500, 1500, 1500, 1500, 1500])      # Capacidad instalada de cada planta con alta capacidad
MBC = np.array([500, 500, 500, 500, 500])           # Capacidad instalada de cada planta con baja capacidad
cost = np.array([[8,14,21,21,12],                   # Costos de distribución
              [14,6,13,14,13],
              [23,13,8,10,22],
              [21,14,9,3,20],
              [12,13,17,20,6]])

In [3]:
# Se definen las variables x (cantidades a distribuir), y (si abre o no la planta en el punto j)
x = cp.Variable((5, 5), nonneg=True)
y = cp.Variable(5, boolean=True)      # Se define como variable indicadora: 0 (BC) o 1 (AC)
z = cp.Variable(5, boolean=True)

# Objetivo:
obj = cp.sum(cp.multiply(AC, y)) + cp.sum(cp.multiply(cost, x)) +cp.sum(cp.multiply(BC, z))

# Restricciones: se verifica que se cumplan las demandas requeridas y que no se exceda la capacidad instalada
constraints = [cp.sum(x[j,:]) <= MAC[j]*y[j]+MBC[j]*z[j] for j in range(5)]
constraints += [cp.sum(x[:,i]) >= d[i] for i in range(5)] 
constraints += [y[j]+z[j] <=1 for j in range(5)] #Esta restricción hace que solo pueda abrirse una planta, o no se abra ninguna.

# Optimización:
prob = cp.Problem(cp.Minimize(obj), constraints) 
prob.solve()

#Solución:
print("Costo Total mínimo: $", prob.value)
print("Plantas que debe abrir en el punto j de Alta Capacidad= ", y.value)
print("Plantas que debe abrir en el punto j de Baja Capacidad= ", z.value)

Costo Total mínimo: $ 58850.899999999994
Plantas que debe abrir en el punto j de Alta Capacidad=  [1. 0. 0. 1. 1.]
Plantas que debe abrir en el punto j de Baja Capacidad=  [0. 0. 1. 0. 0.]


In [4]:
cm1=prob.value

Teniendo en cuenta las demandas dadas, para minimizar los costos, se debe cerrar la planta B. La C debe ser de Baja Capacidad y la A, D y E deben ser de Alta Capacidad.

In [27]:
print('Se tiene una optimización de costos con un valor de',cm1,',y las conclusiones de que se debe abrir la planta A, D, E en alta capacidad, la C de baja capacidad y la planat B cerrada')

Se tiene una optimización de costos con un valor de 58850.899999999994 ,y las conclusiones de que se debe abrir la planta A, D, E en alta capacidad, la C de baja capacidad y la planat B cerrada


In [5]:
from scipy.stats import uniform

Ahora bien, se modela un total de mil demandas bajo una distribución uniforme para cada uno de los puntos. 

In [18]:
dem1 = np.random.uniform(140.4,150.4,10000)
#plt.hist(dem1);

In [19]:
dem2 = np.random.uniform(79.1,89.1,10000)
#plt.hist(dem2);

In [20]:
dem3 = np.random.uniform(151.4,161.4,10000)
#plt.hist(dem3);

In [21]:
dem4 = np.random.uniform(1671.8,1681.8,10000)
#plt.hist(dem4);

In [22]:
dem5 = np.random.uniform(2714.6,2724.6,1000)
#plt.hist(dem5);

In [23]:
# Inputs del problema de optimización
I = np.array([1, 2, 3, 4, 5])                       # Puntos de demanda
J = np.array([1, 2, 3, 4, 5])                       # Plantas
d = np.array([dem1,dem2,dem3,dem4,dem5])            # Demandas a satisfacer
AC = np.array([4730, 7270, 3080, 9100, 9500])       # Costo fijo de cada planta con Alta capacidad
BC = np.array([3230, 4980, 2110, 6230, 6500])       # Costo fijo de cada planta con Baja capacidad
MAC = np.array([1500, 1500, 1500, 1500, 1500])      # Capacidad instalada de cada planta con alta capacidad
MBC = np.array([500, 500, 500, 500, 500])           # Capacidad instalada de cada planta con baja capacidad
cost = np.array([[8,14,21,21,12],                   # Costos de distribución
              [14,6,13,14,13],
              [23,13,8,10,22],
              [21,14,9,3,20],
              [12,13,17,20,6]])

<ipython-input-23-e2c5b35d89b0>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  d = np.array([dem1,dem2,dem3,dem4,dem5])            # Demandas a satisfacer


In [24]:
solt = np.zeros(100)
for i in range(100):
  d=np.array([dem1[i],dem2[i],dem3[i],dem4[i],dem5[i]])
  
  x = cp.Variable((5, 5), nonneg=True)
  y = cp.Variable(5, boolean=True)      # Se define como variable indicadora: 0 (BC) o 1 (AC)
  z = cp.Variable(5, boolean=True)

  # Objetivo:
  obj = cp.sum(cp.multiply(AC, y)) + cp.sum(cp.multiply(cost, x)) +cp.sum(cp.multiply(BC, z))

  # Restricciones: se verifica que se cumplan las demandas requeridas y que no se exceda la capacidad instalada
  constraints = [cp.sum(x[j,:]) <= MAC[j]*y[j]+MBC[j]*z[j] for j in range(5)]
  constraints += [cp.sum(x[:,i]) >= d[i] for i in range(5)] 
  constraints += [y[j]+z[j] <=1 for j in range(5)]

  # Optimización:
  prob = cp.Problem(cp.Minimize(obj), constraints) 
  prob.solve()

  #Solución:
  #print("Costo Total mínimo: $", prob.value)
  #print("Plantas que debe abrir en el punto j de Alta Capacidad= ", y.value)
  #print("Plantas que debe abrir en el punto j de Baja Capacidad= ", z.value)
  sol=prob.value
  solt[i]=sol
 # print(sol)

In [25]:
cm2=solt.mean()

In [26]:
print('En conclusión, al optimizar el problema de costos y la apertura o cierre de cada tipo de planta con las demandas ya dadas,obtenemos una minimización de costos equivalente a',cm1,)
print('. Ahora bien, al modelar las demandas de cada punto, realizar la optimización para cada una de estas y sacar su media, se obtiene un valor bastante similar al inicial de',cm2,)
      

En conclusión, al optimizar el problema de costos y la apertura o cierre de cada tipo de planta con las demandas ya dadas,obtenemos una minimización de costos equivalente a 58850.899999999994
. Ahora bien, al modelar las demandas de cada punto, realizar la optimización para cada una de estas y sacar su media, se obtiene un valor bastante similar al inicial de 58846.268394632054
